# K-fold Cross Validation 을 이용한 집값 예측 

In [87]:
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np
from sklearn.model_selection import train_test_split
from torch import nn,optim
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error

In [34]:
data = pd.read_csv('data/reg.csv',index_col = 0)
data

,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11,x12,x13,Price
0,0.034633,0.206919,0.137057,0.540526,0.193941,0.699239,0.630532,0.239410,0.027375,0.209857,0.347609,0.996394,0.102644,0.422222
1,0.028920,0.014315,0.276113,0.255945,0.618886,0.555407,0.782263,0.482977,0.103031,0.106690,0.520776,0.996650,0.187120,0.368889
2,0.020627,0.033230,0.281116,0.525591,0.165269,0.624102,0.586005,0.272713,0.036010,0.106986,0.595301,0.983284,0.084079,0.660000
3,0.022749,0.033801,0.125044,0.263253,0.251509,0.658532,0.432160,0.344932,0.150018,0.068317,0.651297,0.989989,0.015990,0.631111
4,0.022148,0.029374,0.121057,0.521126,0.399670,0.448086,0.520158,0.495342,0.104383,0.069360,0.560116,0.998723,0.092782,0.693333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
501,0.028702,0.019528,0.455716,0.097575,0.576859,0.588769,0.654701,0.188444,0.007595,0.165409,0.736795,0.982923,0.219891,0.386667
502,0.034217,0.009498,0.490485,0.372934,0.532351,0.580505,0.750547,0.143776,0.051186,0.166428,0.867950,0.995114,0.207453,0.346667
503,0.035352,0.021495,0.423918,0.397988,0.349407,0.610529,0.907637,0.087385,0.083448,0.164870,0.782704,0.995791,0.094044,0.420000
504,0.026182,0.028603,0.443442,0.509663,0.229142,0.667841,0.867135,0.236241,0.016177,0.167554,0.749186,0.986855,0.107399,0.377778


In [35]:
x = data.drop('Price',axis=1).to_numpy()
y = data['Price'].to_numpy().reshape(-1,1)
print(x.shape)
print(y.shape)

(506, 13)
(506, 1)


In [47]:
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size = 0.3)
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(354, 13)
(354, 1)
(152, 13)
(152, 1)


In [42]:
class my_ds(Dataset):
    def __init__(self, x,y):
        self.x = torch.FloatTensor(x)
        self.y = torch.FloatTensor(y)
        self.len = self.y.size()[0]

    def __getitem__(self,index):
        return self.x[index],self.y[index]

    def __len__(self):
        return self.len

In [50]:
traindataset = my_ds(x_train,y_train)
testdataset = my_ds(x_test,y_test)

In [51]:
print(traindataset.x.size())
print(traindataset.y.size())
print(testdataset.x.size())
print(testdataset.y.size())

torch.Size([354, 13])
torch.Size([354, 1])
torch.Size([152, 13])
torch.Size([152, 1])


In [52]:
train_dataloader = DataLoader(traindataset,batch_size = 32,shuffle = True)
test_dataloader = DataLoader(testdataset, batch_size = 32,shuffle = False)

In [76]:
class Regressor(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(13,50)
        self.fc2 = nn.Linear(50,30)
        self.fc3 = nn.Linear(30,1)

    def forward(self,x):
        x = self.fc1(x)
        x = self.fc2(x)
        x = self.fc3(x)
        return x 

In [81]:
kfold = KFold(n_splits = 3,shuffle = True)
criterion = nn.MSELoss()

In [88]:
def Eval(dataloader):

    predictions = torch.tensor([],dtype = torch.float)
    actual = torch.tensor([],dtype = torch.float)

    with torch.no_grad():
        model.eval()
        for data in dataloader:
            x,y = data
            pred = model(x)
            predictions = torch.cat([predictions,pred],0)
            actual = torch.cat([actual,y],0)
    
    predictions = predictions.numpy()
    actual = actual.numpy()
    rmse = np.sqrt(mean_squared_error(predictions,actual))
    model.train()
    return rmse 

In [90]:
for fold,(train_idx,val_idx) in enumerate(kfold.split(traindataset)):
    print('ford:',fold)
    print('train_idx:',train_idx)
    print('val_idx:',val_idx)

ford: 0
train_idx: [  1   2   3   4   5   6   7   9  10  11  12  13  14  16  17  18  19  23
  24  26  29  30  31  32  33  34  36  39  40  41  43  44  45  46  47  48
  51  52  53  54  59  61  62  63  65  66  69  71  75  76  77  78  80  84
  86  87  88  89  90  91  93  94  95  97  98  99 100 101 102 104 105 107
 109 111 112 113 115 116 117 118 120 122 123 124 126 127 130 131 133 134
 135 136 143 144 145 147 148 149 152 153 154 155 156 157 158 163 165 166
 168 170 172 174 175 176 177 179 181 182 183 184 185 186 189 190 192 193
 194 195 196 197 198 200 201 202 203 204 206 209 210 211 212 213 214 215
 217 218 219 222 223 224 226 228 229 232 233 234 235 238 239 240 243 245
 246 248 250 252 254 255 256 257 259 260 261 262 263 264 265 267 268 270
 271 272 273 274 275 278 283 284 289 290 292 293 295 296 297 298 299 300
 301 302 306 307 308 309 311 314 315 316 317 318 319 320 321 322 323 325
 328 329 330 331 333 334 335 336 337 338 339 340 343 344 345 346 347 350
 352 353]
val_idx: [  0   8  15 

In [98]:
validation_loss = []
for fold,(train_idx,val_idx) in enumerate(kfold.split(traindataset)):

    train_subsampler = torch.utils.data.SubsetRandomSampler(train_idx)
    val_subsampler = torch.utils.data.SubsetRandomSampler(val_idx)
    train_loader = DataLoader(traindataset,batch_size = 32, sampler = train_subsampler)
    val_loader = DataLoader(traindataset,batch_size = 32, sampler = val_subsampler)

    model = Regressor()
    optimizer = optim.Adam(model.parameters(),lr = 0.001, weight_decay=1e-7)

    for epoch in range(400):
        for data in train_loader:
            x,y = data
            optimizer.zero_grad()

            output = model(x)
            loss = criterion(output,y)
            loss.backward()
            optimizer.step()

    train_rmse = Eval(train_loader)
    val_rmse = Eval(val_loader)
    print('k-fold',fold,'Train Loss: %.4f, Validation Loss: %4f'%(train_rmse,val_rmse))
    validation_loss.append(val_rmse)

k-fold 0 Train Loss: 0.1276, Validation Loss: 0.117605
k-fold 1 Train Loss: 0.1180, Validation Loss: 0.140682
k-fold 2 Train Loss: 0.1160, Validation Loss: 0.147802


In [102]:
mean = np.mean(validation_loss)
std = np.std(validation_loss)
print(f"Validation Score : {mean} +- {std}")

Validation Score : 0.13536302745342255 +- 0.0128888338804245
